In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import os

In [2]:
spark = SparkSession.builder.getOrCreate()

23/10/20 17:32:55 WARN Utils: Your hostname, MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.1.10 instead (on interface en0)
23/10/20 17:32:55 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/10/20 17:32:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


<h2>1. Đọc data từ nguồn </h2>

In [3]:
input_path = '/Users/habaokhanh/Study_BigData_Dataset/log_content/'

In [4]:
list_file = os.listdir(input_path)
list_file = sorted(list_file)

if os.path.exists(f'{list_file}.DS_Store'):
    os.remove(f'{list_file}.DS_Store')

list_file

['20220401.json',
 '20220402.json',
 '20220403.json',
 '20220404.json',
 '20220405.json',
 '20220406.json',
 '20220407.json',
 '20220408.json',
 '20220409.json',
 '20220410.json',
 '20220411.json',
 '20220412.json',
 '20220413.json',
 '20220414.json',
 '20220415.json',
 '20220416.json',
 '20220417.json',
 '20220418.json',
 '20220419.json',
 '20220420.json',
 '20220421.json',
 '20220422.json',
 '20220423.json',
 '20220424.json',
 '20220425.json',
 '20220426.json',
 '20220427.json',
 '20220428.json',
 '20220429.json',
 '20220430.json']

23/10/20 17:33:11 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [5]:
file_name = list_file[0]
file_name

'20220401.json'

In [6]:
df = spark.read.json(f'{input_path}{file_name}')

<h2>2. Đưa appname về các category </h2>

In [7]:
df.show(truncate=False)

+--------------------+-------+------+--------------------------------------+-----+
|_id                 |_index |_score|_source                               |_type|
+--------------------+-------+------+--------------------------------------+-----+
|AX_momhia1FFivsGrn9o|history|0     |{KPLUS, HNH579912, 0C96E62FC55C, 254} |kplus|
|AX_momhca1FFivsGrnvg|history|0     |{KPLUS, HUFD40665, CCEDDC333614, 1457}|kplus|
|AX_momhaa1FFivsGrnny|history|0     |{KPLUS, HNH572635, B068E6A1C5F6, 2318}|kplus|
|AX_momhca1FFivsGrnvv|history|0     |{KPLUS, HND141717, 08674EE8D2C2, 1452}|kplus|
|AX_momhia1FFivsGrn98|history|0     |{KPLUS, HNH743103, 402343C25D7D, 251} |kplus|
|AX_momg9a1FFivsGrnkS|history|0     |{KPLUS, HNH893773, B84DEE76D3B8, 924} |kplus|
|AX_momhca1FFivsGrnwA|history|0     |{KPLUS, HND083642, B84DEE849A0F, 1444}|kplus|
|AX_momhfa1FFivsGrn2u|history|0     |{KPLUS, DNFD74404, 90324BB44C39, 691} |kplus|
|AX_momhca1FFivsGrnwP|history|0     |{KPLUS, DTFD21200, B84DEED27709, 1436}|kplus|
|AX_

In [8]:
df.printSchema()

root
 |-- _id: string (nullable = true)
 |-- _index: string (nullable = true)
 |-- _score: long (nullable = true)
 |-- _source: struct (nullable = true)
 |    |-- AppName: string (nullable = true)
 |    |-- Contract: string (nullable = true)
 |    |-- Mac: string (nullable = true)
 |    |-- TotalDuration: long (nullable = true)
 |-- _type: string (nullable = true)



In [9]:
df = df.select('_source.*')

In [10]:
df.show()

+-------+---------+------------+-------------+
|AppName| Contract|         Mac|TotalDuration|
+-------+---------+------------+-------------+
|  KPLUS|HNH579912|0C96E62FC55C|          254|
|  KPLUS|HUFD40665|CCEDDC333614|         1457|
|  KPLUS|HNH572635|B068E6A1C5F6|         2318|
|  KPLUS|HND141717|08674EE8D2C2|         1452|
|  KPLUS|HNH743103|402343C25D7D|          251|
|  KPLUS|HNH893773|B84DEE76D3B8|          924|
|  KPLUS|HND083642|B84DEE849A0F|         1444|
|  KPLUS|DNFD74404|90324BB44C39|          691|
|  KPLUS|DTFD21200|B84DEED27709|         1436|
|  KPLUS|LDFD05747|0C96E6C95E53|         1434|
|  KPLUS|HNH063566|B84DEEDD1C85|          687|
|  KPLUS|HNH866786|10394E2790A5|          248|
|  KPLUS|NBAAA1128|10394E47C1AF|          247|
|  KPLUS|HNH960439|B84DEED34371|          683|
|  KPLUS|HNJ035736|CCD4A1FA86A5|          246|
|  KPLUS|NTFD93673|B84DEEEF4763|         2288|
|  KPLUS|HNJ063267|10394E172CA7|         2282|
|  KPLUS|HNH790383|4CEBBD53378B|          906|
|  KPLUS|THFD

In [11]:
from pyspark.sql.functions import *

In [12]:
df = df.withColumn("Type",
       when((col("AppName") == 'CHANNEL') | (col("AppName") =='DSHD')| (col("AppName") =='KPLUS')| (col("AppName") =='KPlus'), "Truyền Hình")
      .when((col("AppName") == 'VOD') | (col("AppName") =='FIMS_RES')| (col("AppName") =='BHD_RES')| 
             (col("AppName") =='VOD_RES')| (col("AppName") =='FIMS')| (col("AppName") =='BHD')| (col("AppName") =='DANET'), "Phim Truyện")
      .when((col("AppName") == 'RELAX'), "Giải Trí")
      .when((col("AppName") == 'CHILD'), "Thiếu Nhi")
      .when((col("AppName") == 'SPORT'), "Thể Thao")
      .otherwise("Error"))

In [13]:
df.show()

+-------+---------+------------+-------------+-----------+
|AppName| Contract|         Mac|TotalDuration|       Type|
+-------+---------+------------+-------------+-----------+
|  KPLUS|HNH579912|0C96E62FC55C|          254|Truyền Hình|
|  KPLUS|HUFD40665|CCEDDC333614|         1457|Truyền Hình|
|  KPLUS|HNH572635|B068E6A1C5F6|         2318|Truyền Hình|
|  KPLUS|HND141717|08674EE8D2C2|         1452|Truyền Hình|
|  KPLUS|HNH743103|402343C25D7D|          251|Truyền Hình|
|  KPLUS|HNH893773|B84DEE76D3B8|          924|Truyền Hình|
|  KPLUS|HND083642|B84DEE849A0F|         1444|Truyền Hình|
|  KPLUS|DNFD74404|90324BB44C39|          691|Truyền Hình|
|  KPLUS|DTFD21200|B84DEED27709|         1436|Truyền Hình|
|  KPLUS|LDFD05747|0C96E6C95E53|         1434|Truyền Hình|
|  KPLUS|HNH063566|B84DEEDD1C85|          687|Truyền Hình|
|  KPLUS|HNH866786|10394E2790A5|          248|Truyền Hình|
|  KPLUS|NBAAA1128|10394E47C1AF|          247|Truyền Hình|
|  KPLUS|HNH960439|B84DEED34371|          683|Truyền Hìn

<h2>3. Summarize data mới ( Group by contract & category & Sum ) </h2>

In [14]:
df.select('AppName','Type').distinct().show()

+-------+-----------+
|AppName|       Type|
+-------+-----------+
|CHANNEL|Truyền Hình|
|  KPLUS|Truyền Hình|
|    VOD|Phim Truyện|
|  CHILD|  Thiếu Nhi|
|  RELAX|   Giải Trí|
|  SPORT|   Thể Thao|
|   FIMS|Phim Truyện|
+-------+-----------+



In [15]:
df.select('Contract', 'Type', 'TotalDuration').show()

+---------+-----------+-------------+
| Contract|       Type|TotalDuration|
+---------+-----------+-------------+
|HNH579912|Truyền Hình|          254|
|HUFD40665|Truyền Hình|         1457|
|HNH572635|Truyền Hình|         2318|
|HND141717|Truyền Hình|         1452|
|HNH743103|Truyền Hình|          251|
|HNH893773|Truyền Hình|          924|
|HND083642|Truyền Hình|         1444|
|DNFD74404|Truyền Hình|          691|
|DTFD21200|Truyền Hình|         1436|
|LDFD05747|Truyền Hình|         1434|
|HNH063566|Truyền Hình|          687|
|HNH866786|Truyền Hình|          248|
|NBAAA1128|Truyền Hình|          247|
|HNH960439|Truyền Hình|          683|
|HNJ035736|Truyền Hình|          246|
|NTFD93673|Truyền Hình|         2288|
|HNJ063267|Truyền Hình|         2282|
|HNH790383|Truyền Hình|          906|
|THFD12466|Truyền Hình|          242|
|HNH566080|Truyền Hình|          242|
+---------+-----------+-------------+
only showing top 20 rows



In [16]:
df.orderBy('Contract').show()

+-------+--------+------------+-------------+-----------+
|AppName|Contract|         Mac|TotalDuration|       Type|
+-------+--------+------------+-------------+-----------+
|CHANNEL|       0|B84DEED2C55B|        16371|Truyền Hình|
|CHANNEL|       0|5CBAEF93B700|           20|Truyền Hình|
|CHANNEL|       0|F8DA0C2A377F|           67|Truyền Hình|
|CHANNEL|       0|10394E17AE5F|           18|Truyền Hình|
|CHANNEL|       0|08674EF3E1B2|        85903|Truyền Hình|
|CHANNEL|       0|10394E17BB8F|           19|Truyền Hình|
|CHANNEL|       0|10394E41DFDC|           63|Truyền Hình|
|CHANNEL|       0|E4AB89210CC9|           20|Truyền Hình|
|CHANNEL|       0|B84DEE78280B|         5806|Truyền Hình|
|CHANNEL|       0|5CEA1D895614|           19|Truyền Hình|
|CHANNEL|       0|D89C67A90C54|           67|Truyền Hình|
|CHANNEL|       0|802BF9C011FE|          437|Truyền Hình|
|CHANNEL|       0|08674EF77C25|        85942|Truyền Hình|
|CHANNEL|       0|B84DEED40692|           19|Truyền Hình|
|CHANNEL|     

In [17]:
df = df.filter(df.Contract != '0')

In [18]:
df.orderBy('Contract', ascending=True).show()

+-------+--------------+------------+-------------+-----------+
|AppName|      Contract|         Mac|TotalDuration|       Type|
+-------+--------------+------------+-------------+-----------+
|  RELAX|113.182.209.48|B046FCA97B6A|           89|   Giải Trí|
|CHANNEL|113.182.209.48|B046FCA97B6A|           20|Truyền Hình|
|CHANNEL|113.182.209.48|B046FCA97B6A|           43|Truyền Hình|
|CHANNEL|     AGAAA0335|10394E3D6D89|        11440|Truyền Hình|
|CHANNEL|     AGAAA0338|10394E3D6E3C|         8895|Truyền Hình|
|CHANNEL|     AGAAA0342|10394E3D7A67|           44|Truyền Hình|
|CHANNEL|     AGAAA0345|10394E3D6E29|        13846|Truyền Hình|
|CHANNEL|     AGAAA0346|10394E3D80EA|        47203|Truyền Hình|
|CHANNEL|     AGAAA0350|10394E3D6EEF|        11090|Truyền Hình|
|CHANNEL|     AGAAA0352|10394E3D857D|        10094|Truyền Hình|
|CHANNEL|     AGAAA0353|10394E3D6EED|           16|Truyền Hình|
|CHANNEL|     AGAAA0356|10394E3D6EEA|        39995|Truyền Hình|
|CHANNEL|     AGAAA0357|10394E3D6D85|   

In [19]:
df.printSchema()

root
 |-- AppName: string (nullable = true)
 |-- Contract: string (nullable = true)
 |-- Mac: string (nullable = true)
 |-- TotalDuration: long (nullable = true)
 |-- Type: string (nullable = false)



<p> Group by contract & category & Sum </p>

In [20]:
df.groupBy("Contract", "Type").agg(sum("TotalDuration").alias("TotalDurationSum")).show()

+---------+-----------+----------------+
| Contract|       Type|TotalDurationSum|
+---------+-----------+----------------+
|DNH014998|Phim Truyện|            3365|
|HND486882|Phim Truyện|            5545|
|HUFD07189|Truyền Hình|            2264|
|HDFD36288|Truyền Hình|           11904|
|CTFD04401|Truyền Hình|           55881|
|HNH954607|Phim Truyện|           13115|
|HNH855959|Truyền Hình|             327|
|SGH034683|Truyền Hình|           82195|
|NTFD35330|Truyền Hình|           19139|
|NTFD48198|Phim Truyện|           55202|
|HNH443856|Truyền Hình|            7687|
|NAFD05338|Truyền Hình|           81934|
|LCFD20510|Phim Truyện|           10852|
|QNFD29007|Truyền Hình|           82705|
|SGH569599|Truyền Hình|           18769|
|SGH701752|Truyền Hình|           31028|
|HNH712164|Phim Truyện|            6106|
|PYFD01920|Truyền Hình|           82664|
|HTFD13716|Truyền Hình|           18759|
|SGJ039473|Truyền Hình|           18695|
+---------+-----------+----------------+
only showing top

<p>Number of contracts of each category and their total duration</p>

In [21]:
df.groupBy("Type").agg(countDistinct("Contract").alias("ContractsCount"),
                       sum("TotalDuration").alias("TotalDurationSum")).show()

+-----------+--------------+----------------+
|       Type|ContractsCount|TotalDurationSum|
+-----------+--------------+----------------+
|   Thể Thao|          2292|         1166238|
|Truyền Hình|       1355366|     24690792816|
|  Thiếu Nhi|         33056|       178570178|
|   Giải Trí|          6394|        20907906|
|Phim Truyện|         86312|       627150472|
+-----------+--------------+----------------+



<h2>4. Pivot data </h2>

<h3>Pivot truyen thong</h3>

In [22]:
TV = df.select('Contract', 'Type', 'TotalDuration').filter(df.Type == 'Truyền Hình')
Movie = df.select('Contract', 'Type', 'TotalDuration').filter(df.Type == 'Phim Truyện')
Relax = df.select('Contract', 'Type', 'TotalDuration').filter(df.Type == 'Giải Trí')
Child = df.select('Contract', 'Type', 'TotalDuration').filter(df.Type == 'Thiếu Nhi')
Sport = df.select('Contract', 'Type', 'TotalDuration').filter(df.Type == 'Thể Thao')

In [23]:
TV = TV.withColumnRenamed('TotalDuration', 'TVDuration').drop('Type')
Movie = Movie.withColumnRenamed('TotalDuration', 'MovieDuration').drop('Type')
Relax = Relax.withColumnRenamed('TotalDuration', 'RelaxDuration').drop('Type')
Child = Child.withColumnRenamed('TotalDuration', 'ChildDuration').drop('Type')
Sport = Sport.withColumnRenamed('TotalDuration', 'SportDuration').drop('Type')

In [24]:
output_manual = TV.join(Movie, on='Contract', how='outer')\
                .join(Relax, on='Contract', how='outer')\
                .join(Child, on='Contract', how='outer')\
                .join(Sport, on='Contract', how='outer')

In [25]:
output_manual.show()

+--------------+----------+-------------+-------------+-------------+-------------+
|      Contract|TVDuration|MovieDuration|RelaxDuration|ChildDuration|SportDuration|
+--------------+----------+-------------+-------------+-------------+-------------+
|113.182.209.48|        20|         null|           89|         null|         null|
|113.182.209.48|        43|         null|           89|         null|         null|
|     AGAAA0338|      8895|         null|         null|         null|         null|
|     AGAAA0342|        44|         null|         null|         null|         null|
|     AGAAA0346|     47203|         null|         null|         null|         null|
|     AGAAA0353|        16|         null|         null|         null|         null|
|     AGAAA0372|        19|         null|         null|         null|         null|
|     AGAAA0504|        11|         null|         null|         null|         null|
|     AGAAA0544|        47|         null|         null|         null|       

In [26]:
output_manual.printSchema()

root
 |-- Contract: string (nullable = true)
 |-- TVDuration: long (nullable = true)
 |-- MovieDuration: long (nullable = true)
 |-- RelaxDuration: long (nullable = true)
 |-- ChildDuration: long (nullable = true)
 |-- SportDuration: long (nullable = true)



In [27]:
output_manual = output_manual.groupBy('Contract').sum()\
                                        .select(col('Contract') \
                                                ,col('sum(TVDuration)').alias('TVDuration')\
                                                ,col('sum(MovieDuration)').alias('MovieDuration') 
                                                ,col('sum(RelaxDuration)').alias('RelaxDuration')
                                                ,col('sum(ChildDuration)').alias('ChildDuration')
                                                ,col('sum(SportDuration)').alias('SportDuration'))

In [28]:
output_manual.filter(output_manual.Contract == '113.182.209.48').show()

+--------------+----------+-------------+-------------+-------------+-------------+
|      Contract|TVDuration|MovieDuration|RelaxDuration|ChildDuration|SportDuration|
+--------------+----------+-------------+-------------+-------------+-------------+
|113.182.209.48|        63|         null|          178|         null|         null|
+--------------+----------+-------------+-------------+-------------+-------------+



In [29]:
output_manual.show()

+---------+----------+-------------+-------------+-------------+-------------+
| Contract|TVDuration|MovieDuration|RelaxDuration|ChildDuration|SportDuration|
+---------+----------+-------------+-------------+-------------+-------------+
|AGAAA0848|        41|         null|         null|         null|         null|
|AGAAA2588|     39375|         null|         null|         null|         null|
|AGD003807|       275|         7074|         null|         null|         null|
|AGD004253|     86400|         null|         null|         null|         null|
|AGD008179|       667|        34907|         null|         null|         null|
|AGD011212|      6087|         null|         null|         null|         null|
|AGD022636|        53|         null|         null|         null|         null|
|AGD026510|     21909|         null|         null|         null|         null|
|AGD029035|     22810|         null|         null|         null|         null|
|AGFD02813|      2854|         7462|         null|  

<h3>Quick pivot</h3>

In [30]:
output =  df.groupBy('Contract').pivot('Type').sum('TotalDuration')

In [31]:
output.show()

+---------+--------+-----------+---------+--------+-----------+
| Contract|Giải Trí|Phim Truyện|Thiếu Nhi|Thể Thao|Truyền Hình|
+---------+--------+-----------+---------+--------+-----------+
|HTFD11598|    null|       2884|     null|    null|        707|
|HPFD48556|      69|       null|     null|    null|      92976|
|NBFD10014|    null|       null|     null|    null|      84628|
|HNH619088|    null|       8456|      234|    null|      65210|
|HNH036174|    null|       null|     null|    null|       6049|
|DNH067877|    null|       null|     null|    null|       5760|
|SGH806190|    null|       null|     null|    null|       1131|
|HNH582022|    null|       null|     null|    null|      86400|
|HNH795510|    null|       5840|     null|    null|      68589|
|DNFD91557|    null|       null|     null|    null|       1640|
|SGH266908|    null|       null|     null|    null|        731|
|HNJ102653|    null|       null|     null|    null|       4586|
|HDFD42710|    null|       null|     nul

In [32]:
output.groupBy("Contract").count().where(col("count")>1).show()

+--------+-----+
|Contract|count|
+--------+-----+
+--------+-----+



In [33]:
output.filter(output.Contract == 'HUD019330').show()

+---------+--------+-----------+---------+--------+-----------+
| Contract|Giải Trí|Phim Truyện|Thiếu Nhi|Thể Thao|Truyền Hình|
+---------+--------+-----------+---------+--------+-----------+
|HUD019330|    null|      10409|     6412|      11|        637|
+---------+--------+-----------+---------+--------+-----------+



In [34]:
output.withColumn('Date', lit('2022-04-01')).show()

+---------+--------+-----------+---------+--------+-----------+----------+
| Contract|Giải Trí|Phim Truyện|Thiếu Nhi|Thể Thao|Truyền Hình|      Date|
+---------+--------+-----------+---------+--------+-----------+----------+
|HTFD11598|    null|       2884|     null|    null|        707|2022-04-01|
|HPFD48556|      69|       null|     null|    null|      92976|2022-04-01|
|NBFD10014|    null|       null|     null|    null|      84628|2022-04-01|
|HNH619088|    null|       8456|      234|    null|      65210|2022-04-01|
|HNH036174|    null|       null|     null|    null|       6049|2022-04-01|
|DNH067877|    null|       null|     null|    null|       5760|2022-04-01|
|SGH806190|    null|       null|     null|    null|       1131|2022-04-01|
|HNH582022|    null|       null|     null|    null|      86400|2022-04-01|
|HNH795510|    null|       5840|     null|    null|      68589|2022-04-01|
|DNFD91557|    null|       null|     null|    null|       1640|2022-04-01|
|SGH266908|    null|     